# Workflow for a multi-regional energy system

In this application of the FINE framework, a multi-regional energy system is modeled and optimized.

All classes which are available to the user are utilized and examples of the selection of different parameters within these classes are given.

The workflow is structures as follows:
1. Required packages are imported and the input data path is set
2. An energy system model instance is created
3. Commodity sources are added to the energy system model
4. Commodity conversion components are added to the energy system model
5. Commodity storages are added to the energy system model
6. Commodity transmission components are added to the energy system model
7. Commodity sinks are added to the energy system model
8. The energy system model is optimized



# 1. Import required packages and set input data path

The FINE framework is imported which provides the required classes and functions for modeling the energy system. The pandas packages is used for import location and time series specific data. The os package is used for setting the input data path.

In [1]:
import FINE as fn
import pandas as pd
import os
from getData import getData

%load_ext autoreload
%autoreload 2

In [2]:
cwd = os.getcwd()
inputDataPath = os.path.join(cwd,"InputData")

In [3]:
data = getData()

# 2. Create an energy system model instance 

The structure of the energy system model is given by the considered locations, commodities, the number of time steps as well as the hours per time step.

The commodities are specified by a unit (i.e. 'GW_electric', 'GW_H2lowerHeatingValue', 'Mio. t CO2/h') which can be given as an energy or mass unit per hour. Furthermore, the cost unit and length unit are specified.

In [4]:
locations = {'cluster_0', 'cluster_1', 'cluster_2', 'cluster_3', 'cluster_4', 'cluster_5', 'cluster_6', 'cluster_7'}
commodityUnitDict = {'electricity': 'GW_el', 'methane': 'GW_CH4_LHV', 'biogas': 'GW_biogas_LHV',
                     'CO2': 'Mio. t CO2/h', 'hydrogen': 'GW_H2_LHV'}
commodities = {'electricity', 'hydrogen', 'methane', 'biogas', 'CO2'}
numberOfTimeSteps=8760
hoursPerTimeStep=1

In [5]:
esM = fn.EnergySystemModel(locations=locations, commodities=commodities, numberOfTimeSteps=8760,
                           commoditiyUnitsDict=commodityUnitDict,
                           hoursPerTimeStep=1, costUnit='1e9 Euro', lengthUnit='km')

In [6]:
CO2_reductionTarget = 1

# 3. Add commodity sources to the energy system model

## 3.1. Electricity sources

### Wind onshore

In [7]:
esM.add(fn.Source(esM=esM, name='Wind (onshore)', commodity='electricity', hasCapacityVariable=True,
                  operationRateMax=data['Wind (onshore), operationRateMax'],
                  capacityMax=data['Wind (onshore), capacityMax'],
                  investPerCapacity=1.1, opexPerCapacity=1.1*0.02, interestRate=0.08,
                  economicLifetime=20))

Full load hours:

In [8]:
data['Wind (onshore), operationRateMax'].sum()

cluster_0    2300.406907
cluster_1    1082.032187
cluster_2    1447.812026
cluster_3    1448.827987
cluster_4    1935.618218
cluster_5    1757.696047
cluster_6    1286.944636
cluster_7    1199.380071
dtype: float64

### Wind offshore

In [9]:
esM.add(fn.Source(esM=esM, name='Wind (offshore)', commodity='electricity', hasCapacityVariable=True,
                  operationRateMax=data['Wind (offshore), operationRateMax'],
                  capacityMax=data['Wind (offshore), capacityMax'],
                  investPerCapacity=2.3, opexPerCapacity=2.3*0.02, interestRate=0.08,
                  economicLifetime=20))

Full load hours:

In [10]:
data['Wind (offshore), operationRateMax'].sum()

cluster_0    4332.977326
cluster_1       0.000000
cluster_2       0.000000
cluster_3       0.000000
cluster_4    3963.044612
cluster_5    3459.283279
cluster_6       0.000000
cluster_7       0.000000
dtype: float64

### PV

In [11]:
esM.add(fn.Source(esM=esM, name='PV', commodity='electricity', hasCapacityVariable=True,
                  operationRateMax=data['PV, operationRateMax'], capacityMax=data['PV, capacityMax'],
                  investPerCapacity=0.65, opexPerCapacity=0.65*0.02, interestRate=0.08,
                  economicLifetime=25))

Full load hours:

In [12]:
data['PV, operationRateMax'].sum()

cluster_0    859.175855
cluster_1    970.535848
cluster_2    913.049274
cluster_3    917.880212
cluster_4    875.448945
cluster_5    904.590003
cluster_6    984.223517
cluster_7    992.311727
dtype: float64

### Exisisting run-of-river hydroelectricity plants

In [13]:
esM.add(fn.Source(esM=esM, name='Existing run-of-river plants', commodity='electricity',
                  hasCapacityVariable=True,
                  operationRateFix=data['Existing run-of-river plants, operationRateFix'],
                  capacityFix=data['Existing run-of-river plants, capacityFix'],
                  investPerCapacity=0, opexPerCapacity=0.208))

## 3.2. Methane (natural gas and biogas)

### Natural gas

In [14]:
esM.add(fn.Source(esM=esM, name='Natural gas purchase', commodity='methane',
                  hasCapacityVariable=False, commodityCost=0.0331*1e-3))

### Biogas

In [15]:
esM.add(fn.Source(esM=esM, name='Biogas purchase', commodity='biogas',
                  operationRateMax=data['Biogas, operationRateMax'], hasCapacityVariable=False,
                  commodityCost=0.05409*1e-3))

## 3.3 CO2

### CO2

The CO2 source is required for the case in which it can be mixed with conventional natural gas.

In [16]:
esM.add(fn.Source(esM=esM, name='CO2 from enviroment', commodity='CO2',
                  hasCapacityVariable=False, commodityLimitID='CO2 limit', yearlyLimit=366*(1-CO2_reductionTarget)))

# 4. Add conversion components to the energy system model

### Biogas to methane

In [17]:
esM.add(fn.Conversion(esM=esM, name='Biogas to methane',
                      commodityConversionFactors={'biogas':-1, 'methane':1, 'CO2':-201*1e-6*0.625},
                      hasCapacityVariable=False))

### Existing combined cycle gas turbine plants

In [18]:
esM.add(fn.Conversion(esM=esM, name='Existing CCGT plants (methane)',
                      commodityConversionFactors={'electricity':1, 'methane':-1/0.625, 'CO2':201*1e-6/0.625},
                      hasCapacityVariable=True, capacityMax=data['Existing CCGT plants (methane), capacityMax'],
                      investPerCapacity=0, opexPerCapacity=0.021, interestRate=0.08,
                      economicLifetime=33))

### New combined cycle gas turbine plants for biogas

In [19]:
esM.add(fn.Conversion(esM=esM, name='New CCGT plants (biogas)',
                      commodityConversionFactors={'electricity':1, 'biogas':-1/0.635},
                      hasCapacityVariable=True, 
                      investPerCapacity=0.7, opexPerCapacity=0.021, interestRate=0.08,
                      economicLifetime=33))

### New combined cycly gas turbines for hydrogen

In [20]:
esM.add(fn.Conversion(esM=esM, name='New CCGT plants (hydrogen)',
                      commodityConversionFactors={'electricity':1, 'hydrogen':-1/0.6},
                      hasCapacityVariable=True, 
                      investPerCapacity=0.927, opexPerCapacity=0.021, interestRate=0.08,
                      economicLifetime=33))

### Electrolyzers

In [21]:
esM.add(fn.Conversion(esM=esM, name='Electroylzers',
                      commodityConversionFactors={'electricity':-1, 'hydrogen':0.7},
                      hasCapacityVariable=True, 
                      investPerCapacity=0.5, opexPerCapacity=0.5*0.025, interestRate=0.08,
                      economicLifetime=10))

# 5. Add commodity storages to the energy system model

## 5.1. Electricity storage

### Lithium ion batteries

The self discharge of a lithium ion battery is here described as 3% per month. The self discharge per hours is obtained using the equation (1-$\text{selfDischarge}_\text{hour})^{30*24\text{h}} = 1-\text{selfDischarge}_\text{month}$.

In [22]:
esM.add(fn.Storage(esM=esM, name='Li-ion batteries', commodity='electricity',
                   hasCapacityVariable=True, chargeEfficiency=0.95,
                   cyclicLifetime=10000, dischargeEfficiency=0.95, selfDischarge=1-(1-0.03)**(1/(30*24)),
                   chargeRate=1, dischargeRate=1,
                   investPerCapacity=0.151, opexPerCapacity=0.002, interestRate=0.08,
                   economicLifetime=22))

## 5.2. Hydrogen storage

### Hydrogen filled salt caverns
The maximum capacity is here obtained by: dividing the given capacity (which is given for methane) by the lower heating value of methane and then multiplying it with the lower heating value of hydrogen.

In [23]:
esM.add(fn.Storage(esM=esM, name='Salt caverns (hydrogen)', commodity='hydrogen',
                   hasCapacityVariable=True, capacityVariableDomain='discrete',
                   capacityPerPlantUnit=133,
                   chargeRate=1/470.37, dischargeRate=1/470.37, sharedPotentialID='Existing salt caverns',
                   stateOfChargeMin=0.33, stateOfChargeMax=1, capacityMax=data['Salt caverns (hydrogen), capacityMax'],
                   investPerCapacity=0.00011, opexPerCapacity=0.00057, interestRate=0.08,
                   economicLifetime=30))

## 5.3. Methane storage

### Methane filled salt caverns

In [24]:
esM.add(fn.Storage(esM=esM, name='Salt caverns (methane)', commodity='methane',
                   hasCapacityVariable=True, capacityVariableDomain='discrete',
                   capacityPerPlantUnit=443,
                   chargeRate=1/470.37, dischargeRate=1/470.37, sharedPotentialID='Existing salt caverns',
                   stateOfChargeMin=0.33, stateOfChargeMax=1, capacityMax=data['Salt caverns (methane), capacityMax'],
                   investPerCapacity=0.00004, opexPerCapacity=0.00001, interestRate=0.08,
                   economicLifetime=30))

## 5.4 Pumped hydro storage

### Pumped hydro storage

In [25]:
esM.add(fn.Storage(esM=esM, name='Pumped hydro storage', commodity='electricity',
                   chargeEfficiency=0.88, dischargeEfficiency=0.88,
                   hasCapacityVariable=True, selfDischarge=1-(1-0.00375)**(1/(30*24)),
                   chargeRate=0.16, dischargeRate=0.12, capacityFix=data['Pumped hydro storage, capacityFix'],
                   investPerCapacity=0, opexPerCapacity=0.000153))

# 6. Add commodity transmission components to the energy system model

## 6.1. Electricity transmission

### AC cables

In [26]:
esM.add(fn.Transmission(esM=esM, name='AC cables', commodity='electricity',
                        hasCapacityVariable=True, capacityFix=data['AC cables, capacityFix']))

The distances of a component are set to a normalized values of 1.


### DC cables

In [27]:
esM.add(fn.Transmission(esM=esM, name='DC cables', commodity='electricity', losses=data['DC cables, losses'],
                        distances=data['DC cables, distances'],
                        hasCapacityVariable=True, capacityFix=data['DC cables, capacityFix']))

## 6.2 Methane transmission

### Methane pipeline

In [28]:
esM.add(fn.Transmission(esM=esM, name='Pipelines (methane)', commodity='methane',
                        distances=data['Pipelines, distances'],
                        hasCapacityVariable=True, hasIsBuiltBinaryVariable=True, bigM=300,
                        locationalEligibility=data['Pipelines, eligibility'],
                        investPerCapacity=0.000037, investIfBuilt=0.000314,
                        interestRate=0.08, economicLifetime=40))

## 6.3 Hydrogen transmission

### Hydrogen pipelines

In [29]:
eligibility=pd.read_excel(os.path.join(inputDataPath,'SpatialData','Pipelines', 'pipelineIncidence.xlsx'),
                          index_col=0, header=0)
distances=pd.read_excel(os.path.join(inputDataPath,'SpatialData','Pipelines', 'pipelineIncidence.xlsx'),
                          index_col=0, header=0)

esM.add(fn.Transmission(esM=esM, name='Pipeline (hydrogen)', commodity='hydrogen',
                        distances=data['Pipelines, distances'],
                        hasCapacityVariable=True, hasIsBuiltBinaryVariable=True, bigM=300,
                        locationalEligibility=data['Pipelines, eligibility'],
                        investPerCapacity=0.000177, investIfBuilt=0.00033,
                        interestRate=0.08, economicLifetime=40))

# 7. Add commodity sinks to the energy system model

## 7.1. Electricity sinks

### Electricity demand

In [30]:
esM.add(fn.Sink(esM=esM, name='Electricity demand', commodity='electricity',
                hasCapacityVariable=False, operationRateFix=data['Electricity demand, operationRateFix']))

## 7.2. Hydrogen sinks

### Fuel cell electric vehicle (FCEV) demand

In [31]:
FCEV_penetration=0.5
esM.add(fn.Sink(esM=esM, name='Hydrogen demand', commodity='hydrogen', hasCapacityVariable=False,
                operationRateFix=data['Hydrogen demand, operationRateFix']*FCEV_penetration))

## 7.3. CO2 sinks

### CO2 exiting the system's boundary

In [32]:
esM.add(fn.Sink(esM=esM, name='CO2 to enviroment', commodity='CO2',
                hasCapacityVariable=False, commodityLimitID='CO2 limit', yearlyLimit=366*(1-CO2_reductionTarget)))

# 8. Optimize energy system model

In [34]:
esM.cluster(numberOfTypicalPeriods=7)
esM.optimize(timeSeriesAggregation=True, optimizationSpecs='LogToConsole=1 OptimalityTol=1e-3 cuts=0 method=2')


Clustering time series data with 7 typical periods and 24 time steps per period...
		(3.3754 sec)

Number of typical periods: 7 Number of time steps per periods: 24
Declaring sets, variables and constraints for SourceSinkModeling
	declaring sets... 
	declaring variables... 
	declaring constraints... 
		(0.3907 sec)
Declaring sets, variables and constraints for ConversionModeling
	declaring sets... 
	declaring variables... 
	declaring constraints... 
		(0.1250 sec)
Declaring sets, variables and constraints for StorageModeling
	declaring sets... 
	declaring variables... 
	declaring constraints... 
		(2.0315 sec)
Declaring sets, variables and constraints for TransmissionModeling
	declaring sets... 
	declaring variables... 
	declaring constraints... 
		(0.3750 sec)
Declaring shared potential constraint...
		(0.0000 sec)
Declaring commodity balances...
		(1.3126 sec)
Declaring objective function...
		(0.6094 sec)
Academic license - for non-commercial use only
Parameter LogToConsole unchang

		(0.8751 sec)


In [35]:
esM._componentModelingDict["SourceSinkModeling"]._capacityVariablesOptimum

,cluster_0,cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6,cluster_7
Existing run-of-river plants,NaN,0.544397,0.455005,0.037456,NaN,0.043572,2.075538,0.655032
PV,17.970141,25.807194,18.698614,32.994484,20.883876,39.127909,22.626522,21.891252
Wind (offshore),16.107011,NaN,NaN,NaN,28.616236,15.276753,NaN,NaN
Wind (onshore),58.169300,0.000000,22.323000,0.000000,32.245161,0.000000,0.000000,0.000000


In [37]:
esM._componentModelingDict["SourceSinkModeling"]._operationVariablesOptimum

0             1          2     \
Biogas purchase      cluster_0   0.000000  0.000000e+00   0.000000   
                     cluster_1   0.859034  8.590345e-01   0.859034   
                     cluster_2   0.834283  8.342827e-01   0.834283   
                     cluster_3   1.313668  1.313668e+00   1.313668   
                     cluster_4   0.000000  0.000000e+00   0.000000   
                     cluster_5   1.256885  1.256885e+00   1.256885   
                     cluster_6   0.989250  9.892501e-01   0.989250   
                     cluster_7   0.885000  8.849996e-01   0.885000   
CO2 from enviroment  cluster_0   0.000000  0.000000e+00   0.000000   
                     cluster_1   0.000000  0.000000e+00   0.000000   
                     cluster_2   0.000000  4.793366e-17   0.000000   
                     cluster_3   0.000000  0.000000e+00   0.000000   
                     cluster_4   0.000000  0.000000e+00   0.000000   
                     cluster_5   0.000000  0.000000e+00   0.000000   
                     cluster_6   0.000000  0.000000e+00   0.000000   
                     cluster_7   0.000000  0.000000e+00   0.000000   
CO2 to enviroment    cluster_0   0.000000  0.000000e+00   0.000000   
                     cluster_1   0.000000  0.000000e+00   0.000000   
                     cluster_2   0.000000  0.000000e+00   0.000000   
                     cluster_3   0.000000  0.000000e+00   0.000000   
                     cluster_4   0.000000  0.000000e+00   0.000000   
                     cluster_5   0.000000  0.000000e+00   0.000000   
                     cluster_6   0.000000  0.000000e+00   0.000000   
                     cluster_7   0.000000  0.000000e+00   0.000000   
Electricity demand   cluster_0   3.775311  3.483287e+00   3.295549   
                     cluster_1  12.887324  1.222285e+01  11.892480   
                     cluster_2  13.321975  1.297251e+01  12.770276   
                     cluster_3   6.226014  5.769065e+00   5.486392   
                     cluster_4   6.298447  5.823279e+00   5.477102   
                     cluster_5   5.373440  5.150210e+00   5.093342   
...                                   ...           ...        ...   
Hydrogen demand      cluster_5   0.185536  1.060219e-01   0.079516   
                     cluster_6   0.149230  8.527496e-02   0.063956   
                     cluster_7   0.161539  9.230875e-02   0.069231   
Natural gas purchase cluster_0   0.000000  0.000000e+00   0.000000   
                     cluster_1   0.000000  0.000000e+00   0.000000   
                     cluster_2   0.000000  0.000000e+00   0.000000   
                     cluster_3   0.000000  0.000000e+00   0.000000   
                     cluster_4   0.000000  0.000000e+00   0.000000   
                     cluster_5   0.000000  0.000000e+00   0.000000   
                     cluster_6   0.000000  0.000000e+00   0.000000   
                     cluster_7   0.000000  0.000000e+00   0.000000   
PV                   cluster_0   0.000000  0.000000e+00   0.000000   
                     cluster_1   0.000000  0.000000e+00   0.000000   
                     cluster_2   0.000000  0.000000e+00   0.000000   
                     cluster_3   0.000000  0.000000e+00   0.000000   
                     cluster_4   0.000000  0.000000e+00   0.000000   
                     cluster_5   0.000000  0.000000e+00   0.000000   
                     cluster_6   0.000000  0.000000e+00   0.000000   
                     cluster_7   0.000000  0.000000e+00   0.000000   
Wind (offshore)      cluster_0  14.165242  1.410115e+01  13.585961   
                     cluster_4  22.285576  2.208602e+01  22.214256   
                     cluster_5   6.245155  6.445064e+00   7.286890   
Wind (onshore)       cluster_0  21.849486  2.269562e+01  23.193483   
                     cluster_1   0.000000  0.000000e+00   0.000000   
                     cluster_2   5.351850  4.472277e+00   4.041921   
                     cluster_3   0.000000  0.00

In [38]:
esM._componentModelingDict["ConversionModeling"]._capacityVariablesOptimum

,cluster_0,cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6,cluster_7
Electroylzers,37.804833,0.000000,0.000000,0.440673,24.865808,1.672738,7.216215e-01,0.000000
Existing CCGT plants (methane),NaN,0.000000,0.000000,NaN,NaN,0.000000,5.843978e-10,0.000000
New CCGT plants (biogas),0.603867,0.545487,0.529770,0.834179,0.734495,0.798122,6.281738e-01,0.561975
New CCGT plants (hydrogen),0.000000,0.788457,9.077589,2.918774,0.000000,2.227679,3.948522e+00,0.000000


In [40]:
esM._componentModelingDict["ConversionModeling"]._capacityVariablesOptimum

,cluster_0,cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6,cluster_7
Electroylzers,37.804833,0.000000,0.000000,0.440673,24.865808,1.672738,7.216215e-01,0.000000
Existing CCGT plants (methane),NaN,0.000000,0.000000,NaN,NaN,0.000000,5.843978e-10,0.000000
New CCGT plants (biogas),0.603867,0.545487,0.529770,0.834179,0.734495,0.798122,6.281738e-01,0.561975
New CCGT plants (hydrogen),0.000000,0.788457,9.077589,2.918774,0.000000,2.227679,3.948522e+00,0.000000


In [41]:
esM._componentModelingDict["StorageModeling"]._capacityVariablesOptimum

,cluster_0,cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6,cluster_7
Li-ion batteries,42.263659,5.097843,8.449528,12.555177,33.688904,31.450115,11.92367,21.775141
Pumped hydro storage,NaN,9.011000,8.265000,19.620000,3.889000,4.018000,9.76500,10.605000
Salt caverns (hydrogen),1197.000000,NaN,11571.000000,11172.000000,19152.000000,266.000000,NaN,NaN
Salt caverns (methane),0.000000,NaN,0.000000,0.000000,0.000000,0.000000,NaN,NaN


In [43]:
esM._componentModelingDict["StorageModeling"]._stateOfChargeOperationVariablesOptimum 

0             1             2     \
Li-ion batteries        cluster_0     27.970887     27.970887  2.797089e+01   
                        cluster_1      0.000000      0.000000  0.000000e+00   
                        cluster_2      2.711904      2.043369  2.043369e+00   
                        cluster_3      0.271553      0.271553  2.715534e-01   
                        cluster_4     26.631503     26.631503  2.663150e+01   
                        cluster_5     25.528729     25.528729  2.552873e+01   
                        cluster_6      3.921607      3.921607  3.921607e+00   
                        cluster_7      0.000000      0.000000  0.000000e+00   
Pumped hydro storage    cluster_1      0.409360      0.409360  4.093601e-01   
                        cluster_2      0.375470      0.375470  3.754701e-01   
                        cluster_3      0.891316      0.891316  8.913157e-01   
                        cluster_4      0.176673      0.176673  1.766731e-01   
                        cluster_5      0.182533      0.182533  1.825335e-01   
                        cluster_6      0.443614      0.443614  4.436135e-01   
                        cluster_7      0.481774      0.481774  4.817738e-01   
Salt caverns (hydrogen) cluster_0   1017.659890   1017.659890  1.017660e+03   
                        cluster_2   8289.858508   8295.837655  8.295838e+03   
                        cluster_3   8303.684810   8303.684810  8.317670e+03   
                        cluster_4  13616.270466  13620.532531  1.362053e+04   
                        cluster_5    200.394854    200.394854  2.003949e+02   
Salt caverns (methane)  cluster_0      0.000000      0.000000  0.000000e+00   
                        cluster_2      0.000000      0.000000  2.384759e-13   
                        cluster_3      0.000000      0.000000  0.000000e+00   
                        cluster_4      0.000000      0.000000  0.000000e+00   
                        cluster_5      0.000000      0.000000  0.000000e+00   

                                           3             4             5     \
Li-ion batteries        cluster_0  2.797089e+01  2.797089e+01  2.797089e+01   
                        cluster_1  0.000000e+00  0.000000e+00  0.000000e+00   
                        cluster_2  2.043369e+00  2.043397e+00  2.043369e+00   
                        cluster_3  2.715534e-01  2.715534e-01  2.715534e-01   
                        cluster_4  2.663150e+01  2.663150e+01  2.663150e+01   
                        cluster_5  2.552873e+01  2.552873e+01  2.552873e+01   
                        cluster_6  1.192367e+01  1.192333e+01  1.192299e+01   
                        cluster_7  0.000000e+00  0.000000e+00  0.000000e+00   
Pumped hydro storage    cluster_1  4.093601e-01  4.093601e-01  4.093601e-01   
                        cluster_2  3.754701e-01  3.754701e-01  3.754701e-01   
                        cluster_3  8.913157e-01  8.913157e-01  8.913157e-01   
                        cluster_4  1.766731e-01  1.766731e-01  1.766731e-01   
                        cluster_5  1.825335e-01  1.825335e-01  1.825335e-01   
                        cluster_6  4.436135e-01  4.436135e-01  4.436135e-01   
                        cluster_7  4.817738e-01  4.817738e-01  4.817738e-01   
Salt caverns (hydrogen) cluster_0  1.020205e+03  1.022750e+03  1.022750e+03   
                        cluster_2  8.295838e+03  8.295838e+03  8.293869e+03   
                        cluster_3  8.322407e+03  8.331315e+03  8.347684e+03   
                        cluster_4  1.361627e+04  1.362230e+04  1.362230e+04   
                        cluster_5  1.998293e+02  1.998293e+02  1.998293e+02   
Salt caverns (methane)  cluster_0  0.000000e+00  0.000000e+00  3.010925e-13   
                        cluster_2  2.384759e-13  2.384759e-13  2.384759e-13   
                        cluster_3  0.000000e+00  1.640910e-12  2.958522e-12   
                        cluster_4  0.000000e+00  0.000000e+00  7.991829e-12   
             

In [46]:
esM._componentModelingDict["TransmissionModeling"]._operationVariablesOptimum

0          1          2     \
cluster_0 AC cables           cluster_3   0.000000   0.000000   0.000000   
                              cluster_4   3.262664   0.000000   0.000000   
                              cluster_5   0.000000   0.000000   0.000000   
          DC cables           cluster_1   0.000000   0.000000   0.000000   
                              cluster_7   0.000000   0.000000   0.000000   
          Pipeline (hydrogen) cluster_4   0.000000   0.000000   0.000000   
          Pipelines (methane) cluster_4   0.000000   0.000000   0.000000   
cluster_1 AC cables           cluster_2   7.558895   7.949758   8.905339   
                              cluster_3   3.453610   4.791545   4.791545   
                              cluster_7   0.000000   9.062097   0.000000   
          DC cables           cluster_0   2.000000   2.000000   2.000000   
                              cluster_2   0.000000   0.000000   0.000000   
          Pipeline (hydrogen) cluster_2   1.941959   1.672878   8.164050   
                              cluster_7   0.000000   0.000000   0.000000   
          Pipelines (methane) cluster_2   0.000000   0.000000   0.000000   
                              cluster_7   0.000000   0.000000   0.000000   
cluster_2 AC cables           cluster_1   0.000000   0.000000   1.205416   
                              cluster_4   8.731961   8.731961   8.731961   
          DC cables           cluster_1   0.000000   0.000000   0.000000   
                              cluster_4   2.000000   2.000000   2.000000   
          Pipeline (hydrogen) cluster_1   0.000000   0.000000   0.000000   
                              cluster_4  21.868421   9.842495  21.868421   
          Pipelines (methane) cluster_1   0.000000   0.000000   0.000000   
                              cluster_4   0.000000   0.000000   0.000000   
cluster_3 AC cables           cluster_0   2.395773   2.395773   2.395773   
                              cluster_1   0.000000   3.376623   0.000000   
                              cluster_4   5.737245   5.737245   5.737245   
                              cluster_5   2.298378  13.381651  13.381651   
                              cluster_6   0.000000   0.000000   0.000000   
                              cluster_7   0.000000   0.000000   0.000000   
...                                            ...        ...        ...   
          Pipelines (methane) cluster_5   0.000000   0.000000   0.000000   
cluster_4 AC cables           cluster_0   0.000000   3.262664   3.262664   
                              cluster_2   0.000000   0.000000   0.000000   
                              cluster_3   0.000000   0.000000   0.000000   
          DC cables           cluster_2   0.000000   0.000000   0.000000   
          Pipeline (hydrogen) cluster_0  18.585294  14.817447  12.407941   
                              cluster_2   8.241218   0.000000   5.614247   
                              cluster_3  17.374475   8.411907  13.412955   
          Pipelines (methane) cluster_0   0.000000   0.000000   0.000000   
                              cluster_2   0.000000   0.000000   0.000000   
                              cluster_3   0.000000   0.000000   0.000000   
cluster_5 AC cables           cluster_0   2.395773   2.395773   2.395773   
                              cluster_3   0.000000  10.660135   9.761440   
                              cluster_6   0.000000   0.000000   0.000000   
          Pipeline (hydrogen) cluster_3   0.185536   0.106022   0.000000   
          Pipelines (methane) cluster_3   0.000000   0.000000   0.000000   
cluster_6 AC cables           cluster_3   0.945700   0.945700   0.644448   
                              cluster_5   1.796830   1.796830   1.796830   
                              cluster_7   6.178572   1.517223   6.178572   
          Pipeline (hydrogen) cluster_7   0.149230   0.085275   6.644826   
          Pipelines (methane) cluster_7   0.000000   0.000000   0.000000   
cluster_7 AC cable